# 用于对本月的历史数据按照板块名称进行分组统计，结果保存到result.csv文件

In [1]:
import pandas as pd
from datetime import datetime
# import sys
# sys.path.append('../utils')
# import constants
from functools import lru_cache
OPTION_DICT = {
    "all": (float('-inf'), float('inf')),
    "0-100": (0, 100),
    "100-500": (100, 500),
    "500-1000": (500, 1000),
    "1000-30000": (1000, 30000),
}

RANGE = ["跌停", "跌<-5%",  "-3%<-5%",     "-3<-1%",
         "平盘", "<3%",     "3-5%",   "5%-涨停", "涨停"]


In [2]:
def get_data() -> tuple[pd.DataFrame, list]:
    """
    获得股票历史信息，并计算总市值
    """
    # 显示结果
    df = pd.read_csv(
        f"../data/merge_{datetime.now().strftime('%Y-%m-%d')}.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    # dates = df.index.unique().sort_values().to_list()
    # print(type(dates[0]))
    # dates = [x.strftime("%Y-%m-%d") for x in dates]
    # 获得当前结果集的日期列表
    # dates_list = [date.strftime('%Y-%m-%d') for date in dates]
    value = pd.read_csv("../data/总股本.csv", index_col=0, dtype={"股票代码": object})
    value_dict = value['总股本'].to_dict()
    df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
    df['总市值'] = df['总股本']*df['收盘']

    return df


In [3]:
print(OPTION_DICT)

{'all': (-inf, inf), '0-100': (0, 100), '100-500': (100, 500), '500-1000': (500, 1000), '1000-30000': (1000, 30000)}


In [4]:
df = get_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 170816 entries, 2023-03-01 to 2023-03-24
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   开盘      170816 non-null  float64
 1   收盘      170816 non-null  float64
 2   最高      170816 non-null  float64
 3   最低      170816 non-null  float64
 4   成交量     170816 non-null  int64  
 5   成交额     170816 non-null  float64
 6   振幅      170816 non-null  float64
 7   涨跌幅     170816 non-null  float64
 8   涨跌额     170816 non-null  float64
 9   换手率     170816 non-null  float64
 10  股票代码    170816 non-null  object 
 11  股票名称    170816 non-null  object 
 12  板块名称    170816 non-null  object 
 13  总股本     163580 non-null  float64
 14  总市值     163580 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 20.9+ MB


In [5]:
df.columns

Index(['开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率', '股票代码',
       '股票名称', '板块名称', '总股本', '总市值'],
      dtype='object')

## 按日期、板块名称分组，并统计涨幅大于0和小于0的股票数量

In [6]:

def get_count(cur_df):
    """
    根据日期,板块名称对涨跌幅进行
    """
    # cur_df = df.copy()
    # 按股票名称分组，并统计涨幅大于0和小于0的股票数量
    result = cur_df.groupby(['日期', '板块名称'])['涨跌幅'].agg(
        [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0))])
    result['涨幅比'] = result['涨的数量']/(result['涨的数量']+result['跌的数量'])*100
    # result.reset_index(inplace=True)
    return result


## 按日期、板块名称分组，并统计涨幅平均值以及总市值求和

In [7]:

def get_sum(cur_df):
    """
    根据start_value, end_value过滤总市值
    """
    value_df = cur_df.groupby(['日期', "板块名称"]).agg(
        {"涨跌幅": "mean", "总市值": "sum"})
    # value_df.reset_index(inplace=True)
    return value_df


## 合并结果

## 按照涨跌幅的区间进行统计

In [8]:

def get_range(cur_df):  # 按涨跌幅统计
      
     # db = df.loc['2023-03-01']
     bins = [-20, -10, -5, -3, -0.099, 0.099, 3, 5, 10, 20]
     # bins = list(range(-11, 12))
     cuts = pd.cut(cur_df['涨跌幅'], bins=bins)
     pct_chg_list = cur_df.groupby(["日期", "板块名称", cuts])['涨跌幅'].count()
     cur_df = pct_chg_list.unstack()
     return cur_df


### 用于测试

In [9]:
cur_df = df.copy()
# cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
#                 & (cur_df['总市值'] <= (end_value)*100_000_000)]
result = get_count(cur_df)
value_df = get_sum(cur_df)
result


涨的数量  跌的数量        涨幅比
日期         板块名称                       
2023-03-01 IT服务   120     7  94.488189
           LED     31     7  81.578947
           专业工程    31     3  91.176471
           专业服务    20     6  76.923077
           专业连锁     6     1  85.714286
...               ...   ...        ...
2023-03-24 饮料制造    29    18  61.702128
           饰品       4    11  26.666667
           饲料      13     4  76.470588
           高速公路     2    17  10.526316
           黑色家电     8     3  72.727273

[5184 rows x 3 columns]

In [10]:
value_df

涨跌幅           总市值
日期         板块名称                        
2023-03-01 IT服务  4.119685  1.235165e+12
           LED   1.099000  2.977415e+11
           专业工程  1.343529  3.725464e+11
           专业服务  0.585556  1.848354e+11
           专业连锁  0.792500  6.418736e+10
...                   ...           ...
2023-03-24 饮料制造  0.271064  4.812782e+12
           饰品   -0.426250  1.216285e+11
           饲料    0.875556  2.813987e+11
           高速公路 -1.489500  2.310754e+11
           黑色家电  1.882727  1.294439e+11

[5184 rows x 2 columns]

In [11]:
final_df = result.join(value_df, on=["日期", "板块名称"])
final_df

涨的数量  跌的数量        涨幅比       涨跌幅           总市值
日期         板块名称                                               
2023-03-01 IT服务   120     7  94.488189  4.119685  1.235165e+12
           LED     31     7  81.578947  1.099000  2.977415e+11
           专业工程    31     3  91.176471  1.343529  3.725464e+11
           专业服务    20     6  76.923077  0.585556  1.848354e+11
           专业连锁     6     1  85.714286  0.792500  6.418736e+10
...               ...   ...        ...       ...           ...
2023-03-24 饮料制造    29    18  61.702128  0.271064  4.812782e+12
           饰品       4    11  26.666667 -0.426250  1.216285e+11
           饲料      13     4  76.470588  0.875556  2.813987e+11
           高速公路     2    17  10.526316 -1.489500  2.310754e+11
           黑色家电     8     3  72.727273  1.882727  1.294439e+11

[5184 rows x 5 columns]

In [12]:
# final_df.dropna(inplace=True, axis=0)
# 将Salary列格式化为亿元
final_df['总市值亿元'] = final_df['总市值'].apply(
    lambda x: '{:.2f}'.format(x/100000000))

db = get_range(cur_df)
result = pd.merge(final_df, db, on=["日期",'板块名称'])
result.reset_index(inplace=True,level=[0,1])
a=result.columns[:8].to_list()
a.extend(RANGE)
result.columns = a


In [13]:
result.columns

Index(['日期', '板块名称', '涨的数量', '跌的数量', '涨幅比', '涨跌幅', '总市值', '总市值亿元', '跌停',
       '跌<-5%', '-3%<-5%', '-3<-1%', '平盘', '<3%', '3-5%', '5%-涨停', '涨停'],
      dtype='object')

## 正式运行

In [14]:
for key,value in OPTION_DICT.items():
    start_value, end_value = value
    cur_df = df.copy()
    cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
                    & (cur_df['总市值'] <= (end_value)*100_000_000)]
    result = get_count(cur_df)
    value_df = get_sum(cur_df)
    final_df = result.join(value_df, on=["日期", "板块名称"])
    # final_df.dropna(inplace=True, axis=0)
    # 将Salary列格式化为亿元
    final_df['总市值亿元'] = final_df['总市值'].apply(
        lambda x: '{:.2f}'.format(x/100000000))
    
    db = get_range(cur_df)
    result = pd.merge(final_df, db, on=["日期",'板块名称'])
    result.reset_index(inplace=True,level=[0,1])
    # 获得字段的前八列
    a = result.columns[:8].to_list()
    a.extend(RANGE)
    result.columns = a

    result.to_csv(
        f"../data/result_{key}_{datetime.now().strftime('%Y%m%d')}.csv", index=False)


## 校验一下数据结果

In [17]:
df = pd.read_csv(
    f"../data/result_all_{datetime.now().strftime('%Y%m%d')}.csv", index_col=0, parse_dates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5184 entries, 2023-03-01 to 2023-03-24
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   板块名称     5184 non-null   object 
 1   涨的数量     5184 non-null   int64  
 2   跌的数量     5184 non-null   int64  
 3   涨幅比      5184 non-null   float64
 4   涨跌幅      5184 non-null   float64
 5   总市值      5184 non-null   float64
 6   总市值亿元    5184 non-null   float64
 7   跌停       5184 non-null   int64  
 8   跌<-5%    5184 non-null   int64  
 9   -3%<-5%  5184 non-null   int64  
 10  -3<-1%   5184 non-null   int64  
 11  平盘       5184 non-null   int64  
 12  <3%      5184 non-null   int64  
 13  3-5%     5184 non-null   int64  
 14  5%-涨停    5184 non-null   int64  
 15  涨停       5184 non-null   int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 688.5+ KB


In [18]:
df.head()

,板块名称,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-3%<-5%,-3<-1%,平盘,<3%,3-5%,5%-涨停,涨停
日期,,,,,,,,,,,,,,,,
2023-03-01,IT服务,116,6,95.081967,4.267131,1.235165e+12,12351.65,0,0,0,5,1,37,45,29,4
2023-03-01,LED,31,7,81.578947,1.127179,2.977415e+11,2977.42,0,0,0,5,4,28,1,1,0
2023-03-01,专业工程,31,3,91.176471,1.343529,3.725464e+11,3725.46,0,0,0,3,1,28,1,1,0
2023-03-01,专业服务,18,6,75.000000,0.582400,1.848354e+11,1848.35,0,0,0,6,4,12,3,0,0
2023-03-01,专业连锁,6,1,85.714286,0.792500,6.418736e+10,641.87,0,1,0,0,1,5,0,1,0


In [19]:
color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]


In [21]:
my_df=df.loc["2023-03-01"]
my_df=my_df[my_df['板块名称'] == "IT服务"]

color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]

my=my_df[RANGE].unstack()
my=my.reset_index()
my.columns=['x',"date","y"]
# my
data = pd.DataFrame({"x": my["x"], "y": my["y"], "color": color})
data

,x,y,color
0,跌停,0,green
1,跌<-5%,0,green
2,-3%<-5%,0,green
3,-3<-1%,5,green
4,平盘,1,yellow
5,<3%,37,red
6,3-5%,45,red
7,5%-涨停,29,red
8,涨停,4,red


In [32]:
from st_aggrid.grid_options_builder import GridOptionsBuilder
import plotly.graph_objs as go
code_df=data
fig = go.Figure([go.Bar(x=code_df['x'], y=code_df['y'], marker={
    'color': code_df["color"]}, text=code_df['y'], textposition='auto')])
fig.update_traces(
    texttemplate='%{text:.2d}', textposition='outside')
fig.update_layout(autosize=True, margin=dict(
    l=70, r=70, t=70, b=70))
fig.update_layout(
    xaxis_title='区间', yaxis_title='数量')

fig.show()

In [ ]:
df.groupby(df.index).sum() 